<a href="https://colab.research.google.com/github/aekanun2020/2022-PUB_COC-Data-Science-for-Tourism/blob/main/DEMO-Spark-BEST_2022_JUN_Prediction_of_Loan_Payment_allAlgorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preconfiguration

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
   .appName("Neural Network Model") \
   .config("spark.executor.memory", "3gb") \
   .getOrCreate()
   
sc = spark.sparkContext

sc

<SparkContext master=local[*] appName=pyspark-shell>

In [2]:
! rm -rf LoanStats_web*

In [3]:
! wget https://storage.googleapis.com/grizzy-lab/LoanStats_web.csv

--2022-09-24 15:19:08--  https://storage.googleapis.com/grizzy-lab/LoanStats_web.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.127.128, 142.250.153.128, 142.250.145.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.127.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1160243241 (1.1G) [text/csv]
Saving to: ‘LoanStats_web.csv’

LoanStats_web.csv   100%[===================>]   1.08G  76.7MB/s    in 18s     

2022-09-24 15:19:27 (63.1 MB/s) - ‘LoanStats_web.csv’ saved [1160243241/1160243241]



In [4]:
import time as t

In [5]:
! pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from pyspark.sql import functions as F

In [9]:
spark

In [10]:
raw_LendingClubWeb_df = spark.read.format('csv').\
option('header','true').option('mode','DROPMALFORMED')\
.load('LoanStats_web.csv')

# 2. Data Understanding

In [11]:
## Get row count
raw_LendingClubWeb_df.count()

1432466

In [12]:
## Get Data Type
raw_LendingClubWeb_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: string (nullable = true)
 |-- funded_amnt: string (nullable = true)
 |-- funded_amnt_inv: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string 

In [13]:
## Get Freq. Distribution of "purpose" (อาจเจอ outlier)
raw_LendingClubWeb_df.groupBy('grade').count().orderBy('count').show(100)

+-----+------+
|grade| count|
+-----+------+
| null|    26|
|    G|  5271|
|    F| 19480|
|    E| 68353|
|    D|191137|
|    A|302988|
|    C|421101|
|    B|424110|
+-----+------+



In [14]:
ALL = raw_LendingClubWeb_df.count()

In [15]:
ALL

1432466

In [16]:
## Get Freq. Distribution of "purpose" (อาจเจอ outlier)
raw_LendingClubWeb_df.groupBy('grade').count()\
.withColumnRenamed('count','gradeCount')\
.withColumn('gradePercentage',(F.col('gradeCount')/ALL)*100)\
.orderBy('gradePercentage').show(100)

+-----+----------+--------------------+
|grade|gradeCount|     gradePercentage|
+-----+----------+--------------------+
| null|        26|0.001815051805767...|
|    G|      5271|  0.3679668487768645|
|    F|     19480|  1.3598926606285944|
|    E|     68353|   4.771701387676916|
|    D|    191137|   13.34321373072729|
|    A|    302988|  21.151496789452594|
|    C|    421101|   29.39692809462842|
|    B|    424110|   29.60698543630355|
+-----+----------+--------------------+



In [17]:
## Check for Imbalance Class
raw_LendingClubWeb_df.select(['loan_status']).groupBy('loan_status').count().orderBy('count').show(100)

+------------------+------+
|       loan_status| count|
+------------------+------+
|              null|    26|
|           Default|  1419|
| Late (16-30 days)|  4986|
|   In Grace Period|  6136|
|Late (31-120 days)| 19455|
|       Charged Off|152999|
|        Fully Paid|583755|
|           Current|663690|
+------------------+------+



In [18]:
## Get data that contain ONLY Fully Paid and Charged Off
loanPayment_df = raw_LendingClubWeb_df\
.filter((F.col('loan_status') == 'Fully Paid') | ((F.col('loan_status') =='Charged Off')))

In [19]:
loanPayment_df.groupBy('loan_status').count().show()

+-----------+------+
|loan_status| count|
+-----------+------+
| Fully Paid|583755|
|Charged Off|152999|
+-----------+------+



In [20]:
## Get data that contain ONLY related attributes with Business Prespective.
business_df = loanPayment_df.select('annual_inc'\
                                           ,'bc_util'\
                                           ,'inq_fi'\
                                           ,'inq_last_12m'\
                                           ,'home_ownership'\
                                           ,'purpose'\
                                           ,'emp_length'\
                                           ,'revol_bal'\
                                           ,'dti'\
                                           ,'delinq_2yrs'\
                                           ,'pub_rec_bankruptcies'\
                                           ,'pub_rec'\
                                           ,'open_rv_24m'\
                                           ,'mort_acc'\
                                           ,'num_actv_bc_tl'\
                                           ,'num_actv_rev_tl'\
                                           ,'num_il_tl'\
                                           ,'num_tl_90g_dpd_24m'\
                                           ,'int_rate'\
                                           ,'inq_last_6mths'\
                                           ,'term'\
                                           ,'installment'\
                                           ,'total_rev_hi_lim'\
                                           ,'total_bal_il'\
                                           ,'total_bal_ex_mort'\
                                           ,'total_acc'\
                                           ,'tot_cur_bal'\
                                           ,'loan_amnt'\
                                           ,'loan_status'\
                                           ,'verification_status'\
                                           ,'collections_12_mths_ex_med'\
                                           ,'chargeoff_within_12_mths'\
                                           ,'il_util'\
                                           ,'last_pymnt_amnt'\
                                           #,'last_pymnt_d'\
                                           ,'out_prncp_inv'\
                                           ,'out_prncp'\
                                           ,'total_pymnt_inv'\
                                           ,'total_pymnt'\
                                           ,'grade')
                                           #,'sec_app_collections_12_mths_ex_med'\
                                           #,'sec_app_chargeoff_within_12_mths'\
                                           #,'settlement_term'\
                                           #,'settlement_amount'\
                                           #,'settlement_status'

In [21]:
len(business_df.columns)

39

In [22]:
## Get Missing, Dispersion and Range (Validity)
import pandas as pd
business_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
annual_inc,736754,79498.4926439083,78094.7518701319,0,99999.84
bc_util,727864,55.10990899398759,28.903095321098565,0,99.9
inq_fi,736698,1.0650497218670336,5.890278008272024,0,9
inq_last_12m,736697,2.2510679424512383,21.116503762814474,0,9
home_ownership,736754,None,None,ANY,RENT
purpose,736754,None,None,and also pay off some credit card debt. I wi...,wedding
emp_length,736754,None,None,1 year,n/a
revol_bal,736753,16101.816257280256,22819.366258659797,0,99991
dti,736282,18.711086881231356,13.79426027607045,-1,Debt consolidation


### สรุปการดำเนินงานของ Data Understanding: เลือกเฉพาะบาง Attributes และ Row ที่เป็น Charge Off/Fully Charge

# 3. Data Preparation

In [23]:
## Get Data with No Null
no_null_df = business_df.dropna(how='any')

In [24]:
### from pyspark.sql.functions import *

In [25]:
## Get Data with No "wedding"
no_wedding_df = no_null_df.filter(F.col('purpose') != 'wedding')

In [26]:
fitmem_no_null_df = no_wedding_df.repartition(60)

In [27]:
cached_no_null_df = fitmem_no_null_df.cache()

In [28]:
## นับจำนวนข้อมูลหลัง Cleansing ค่า 'Null'
cached_no_null_df.count()

627709

### Remove '%' และ Extract Month

In [29]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

### Remove '%' 

In [30]:
def f_removepercent(origin):
    return origin.rstrip('%')

In [31]:
removepercent = udf(lambda x: f_removepercent(x),StringType())

### Extract Month

In [32]:
def f_exrtractmonth(origin):
    return origin.split('-')[0]

In [33]:
exrtractmonth = udf(lambda x: f_exrtractmonth(x),StringType())

### Replace'n/a' ใน Field 'emp_length' ด้วย 'Notemployed'

In [34]:
def python_treatNA(origin):
    if origin == 'n/a':
        new = 'NotEmployed'
    else:
        new = origin
    return new

In [35]:
treatNA = udf(lambda x: python_treatNA(x),StringType())

### เปลี่ยนแปลง Type ของข้อมูล

In [36]:
from pyspark.sql.functions import col

In [37]:
crunched_df = cached_no_null_df.\
withColumn('emp_length',treatNA(cached_no_null_df['emp_length'])).\
withColumn('int_rate',removepercent(cached_no_null_df['int_rate']).cast(DoubleType())).\
withColumn('dti',cached_no_null_df['dti'].cast(DoubleType())).\
withColumn('revol_bal',cached_no_null_df['revol_bal'].cast(DoubleType())).\
withColumn('pub_rec',cached_no_null_df['pub_rec'].cast(DoubleType())).\
withColumn('total_bal_il',cached_no_null_df['total_bal_il'].cast(DoubleType())).\
withColumn('tot_cur_bal',cached_no_null_df['tot_cur_bal'].cast(DoubleType())).\
withColumn('total_acc',cached_no_null_df['total_acc'].cast(DoubleType())).\
withColumn('total_bal_ex_mort',cached_no_null_df['total_bal_ex_mort'].cast(DoubleType())).\
withColumn('total_rev_hi_lim',cached_no_null_df['total_rev_hi_lim'].cast(DoubleType())).\
withColumn('num_actv_rev_tl',cached_no_null_df['num_actv_rev_tl'].cast(DoubleType())).\
withColumn('num_actv_bc_tl',cached_no_null_df['num_actv_bc_tl'].cast(DoubleType())).\
withColumn('num_il_tl',cached_no_null_df['num_il_tl'].cast(DoubleType())).\
withColumn('pub_rec_bankruptcies',cached_no_null_df['pub_rec_bankruptcies'].cast(DoubleType())).\
withColumn('delinq_2yrs',cached_no_null_df['delinq_2yrs'].cast(DoubleType())).\
withColumn('open_rv_24m',cached_no_null_df['open_rv_24m'].cast(DoubleType())).\
withColumn('num_tl_90g_dpd_24m',cached_no_null_df['num_tl_90g_dpd_24m'].cast(DoubleType())).\
withColumn('inq_last_6mths',cached_no_null_df['inq_last_6mths'].cast(DoubleType())).\
withColumn('bc_util',cached_no_null_df['bc_util'].cast(DoubleType())).\
withColumn('mort_acc',cached_no_null_df['mort_acc'].cast(DoubleType())).\
withColumn('inq_fi',cached_no_null_df['inq_fi'].cast(DoubleType())).\
withColumn('last_pymnt_amnt',cached_no_null_df['last_pymnt_amnt'].cast(DoubleType())).\
withColumn('out_prncp_inv',cached_no_null_df['out_prncp_inv'].cast(DoubleType())).\
withColumn('out_prncp',cached_no_null_df['out_prncp'].cast(DoubleType())).\
withColumn('total_pymnt_inv',cached_no_null_df['total_pymnt_inv'].cast(DoubleType())).\
withColumn('total_pymnt',cached_no_null_df['total_pymnt'].cast(DoubleType())).\
withColumn('il_util',cached_no_null_df['il_util'].cast(DoubleType())).\
withColumn('chargeoff_within_12_mths',cached_no_null_df['chargeoff_within_12_mths'].cast(DoubleType())).\
withColumn('collections_12_mths_ex_med',cached_no_null_df['collections_12_mths_ex_med'].cast(DoubleType())).\
withColumn('loan_amnt',cached_no_null_df['loan_amnt'].cast(DoubleType())).\
withColumn('inq_last_12m',cached_no_null_df['inq_last_12m'].cast(DoubleType())).\
withColumn('installment',cached_no_null_df['installment'].cast(DoubleType())).\
withColumn('annual_inc',cached_no_null_df['annual_inc'].cast(DoubleType()))

In [38]:
crunched_df.printSchema()

root
 |-- annual_inc: double (nullable = true)
 |-- bc_util: double (nullable = true)
 |-- inq_fi: double (nullable = true)
 |-- inq_last_12m: double (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- revol_bal: double (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: double (nullable = true)
 |-- pub_rec_bankruptcies: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- open_rv_24m: double (nullable = true)
 |-- mort_acc: double (nullable = true)
 |-- num_actv_bc_tl: double (nullable = true)
 |-- num_actv_rev_tl: double (nullable = true)
 |-- num_il_tl: double (nullable = true)
 |-- num_tl_90g_dpd_24m: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- inq_last_6mths: double (nullable = true)
 |-- term: string (nullable = true)
 |-- installment: double (nullable = true)
 |-- total_rev_hi_lim: double (nullable = true)
 |-- total_bal_

In [39]:
## ทดลองความถูกต้องของ 'emp_length' หลังจาก Replacement แล้ว
crunched_df.select(crunched_df['emp_length']).distinct().show()

+-----------+
| emp_length|
+-----------+
|    5 years|
|    9 years|
|     1 year|
|NotEmployed|
|    2 years|
|    7 years|
|    8 years|
|    4 years|
|    6 years|
|    3 years|
|  10+ years|
|   < 1 year|
+-----------+



In [40]:
crunched_df.select(crunched_df['purpose']).distinct().show()

+------------------+
|           purpose|
+------------------+
|             other|
|    small_business|
|debt_consolidation|
|       credit_card|
|            moving|
|          vacation|
|  renewable_energy|
|             house|
|               car|
|    major_purchase|
|           medical|
|  home_improvement|
+------------------+



### Normalization 'annual_inc'

In [41]:
from pyspark.sql.functions import *
max_annual_inc = crunched_df.select(max('annual_inc')).collect()[0][0]
min_annual_inc = crunched_df.select(min('annual_inc')).collect()[0][0]

In [42]:
def t_annual_inc(origin):
    return ((origin-min_annual_inc)/(max_annual_inc-min_annual_inc))

In [43]:
n_annual_inc = udf(lambda x: t_annual_inc(x),DoubleType())

### Normalization 'revol_bal'

In [44]:
max_revol_bal = crunched_df.select(max('revol_bal')).collect()[0][0]
min_revol_bal = crunched_df.select(min('revol_bal')).collect()[0][0]

In [45]:
def t_revol_bal(origin):
    return ((origin-min_revol_bal)/(max_revol_bal-min_revol_bal))

In [46]:
n_revol_bal = udf(lambda x: t_revol_bal(x),DoubleType())

### Normalization 'tot_cur_bal'

In [47]:
max_tot_cur_bal = crunched_df.select(max('tot_cur_bal')).collect()[0][0]
min_tot_cur_bal = crunched_df.select(min('tot_cur_bal')).collect()[0][0]

In [48]:
def t_tot_cur_bal(origin):
    return ((origin-min_tot_cur_bal)/(max_tot_cur_bal-min_tot_cur_bal))

In [49]:
n_tot_cur_bal = udf(lambda x: t_tot_cur_bal(x),DoubleType())

### Normalization 'total_rev_hi_lim'

In [50]:
max_total_rev_hi_lim = crunched_df.select(max('total_rev_hi_lim')).collect()[0][0]
min_total_rev_hi_lim = crunched_df.select(min('total_rev_hi_lim')).collect()[0][0]

In [51]:
def t_total_rev_hi_lim(origin):
    return ((origin-min_total_rev_hi_lim)/(max_total_rev_hi_lim-min_total_rev_hi_lim))

In [52]:
n_total_rev_hi_lim = udf(lambda x: t_total_rev_hi_lim(x),DoubleType())

### Normalization 'total_bal_ex_mort'

In [53]:
max_total_bal_ex_mort = crunched_df.select(max('total_bal_ex_mort')).collect()[0][0]
min_total_bal_ex_mort = crunched_df.select(min('total_bal_ex_mort')).collect()[0][0]

In [54]:
def t_total_bal_ex_mort(origin):
    return ((origin-min_total_bal_ex_mort)/(max_total_bal_ex_mort-min_total_bal_ex_mort))

In [55]:
n_total_bal_ex_mort = udf(lambda x: t_total_bal_ex_mort(x),DoubleType())

### Normalization 'total_bal_il'

In [56]:
max_total_bal_il = crunched_df.select(max('total_bal_il')).collect()[0][0]
min_total_bal_il = crunched_df.select(min('total_bal_il')).collect()[0][0]

In [57]:
def t_total_bal_il(origin):
    return ((origin-min_total_bal_il)/(max_total_bal_il-min_total_bal_il))

In [58]:
n_total_bal_il = udf(lambda x: t_total_bal_il(x),DoubleType())

In [59]:
## Call functions of data normalization
normalized_df = crunched_df.\
withColumn('annual_inc',n_annual_inc(crunched_df['annual_inc'])).\
withColumn('revol_bal',n_revol_bal(crunched_df['revol_bal'])).\
withColumn('tot_cur_bal',n_tot_cur_bal(crunched_df['tot_cur_bal'])).\
withColumn('total_rev_hi_lim',n_total_rev_hi_lim(crunched_df['total_rev_hi_lim'])).\
withColumn('total_bal_il',n_total_bal_il(crunched_df['total_bal_il'])).\
withColumn('total_bal_ex_mort',n_total_bal_ex_mort(crunched_df['total_bal_ex_mort']))

#withColumn('loan_amnt',n_loan_amt(crunched_df['loan_amnt'])).\

In [60]:
normalized_df.count()

627709

In [61]:
normalized_filtered_df = normalized_df

In [ ]:
## ตรวจดู Null, Dispersion และ Range (Validity)
normalized_df.describe().toPandas().transpose()

### กำจัดค่า 'Null' จากการทำ Data Prep

In [ ]:
data_no_missing_df = normalized_filtered_df.dropna(how='any')

In [ ]:
data_no_missing_df.count()

In [ ]:
data_no_missing_df.printSchema()

# 4. Modeling

### แบ่งข้อมูลเป็น Training และ Testing

In [ ]:
#แบ่งข้อมูลเป็น Training และ Testing (ุ60% - 40%) โดย Random จากข้อมูลที่เหลืออยู่

In [ ]:
training_dt, test_dt = normalized_df.randomSplit([0.6,0.4], seed = 13)

In [ ]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.feature import StringIndexer, VectorAssembler, \
OneHotEncoder, VectorIndexer
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, \
BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression, GBTClassifier, \
NaiveBayes, RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.ml.clustering import *

### Feature Transformation - Convert from Categorical Vars to Numerical ones

In [ ]:
labelIndexer = StringIndexer(inputCol='loan_status',outputCol='indexedLabel')

In [ ]:
gradeIndexer = StringIndexer(inputCol='grade',outputCol='gradeIndexed')
gradeOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='gradeIndexed',\
                                  outputCol='gradeVec')

In [ ]:
homeIndexer = StringIndexer(inputCol='home_ownership',outputCol='homeIndexed')
homeOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='homeIndexed',\
                                  outputCol='homeVec')

In [ ]:
purposeIndexer = StringIndexer(inputCol='purpose',outputCol='purposeIndexed')
purposeOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='purposeIndexed',\
                                  outputCol='purposeVec')

In [ ]:
emp_lengthIndexer = StringIndexer(inputCol='emp_length',outputCol='emp_lengthIndexed')
emp_lengthOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='emp_lengthIndexed',\
                                  outputCol='emp_lengthVec')

In [ ]:
verification_statusIndexer = StringIndexer(inputCol='verification_status',outputCol='verification_statusIndexed')
verification_statusOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='verification_statusIndexed',\
                                  outputCol='verification_statusVec')

### [FINAL] Feature Selection 

In [ ]:
featureAssembler = VectorAssembler(inputCols=['annual_inc'\
                                              #,'bc_util'\
                                              #,'inq_fi'\
                                              #,'inq_last_12m'\
                                              #,'home_ownership'\
                                              #,'purpose'\
                                              #,'emp_length'\
                                              ,'installment'\
                                              #,'total_rev_hi_lim'\
                                              ,'loan_amnt'\
                                              #,'loan_status'\
                                              #,'verification_status'\
                                              ,'total_pymnt'\
                                              ,'gradeVec'\
                                              ,'homeVec'\
                                              ,'emp_lengthVec'\
                                              ,'purposeVec'\
                                              ,'verification_statusVec']\
                                   ,outputCol='***features')

In [ ]:
selected_attr_list = ['annual_inc'\
                                              #,'bc_util'\
                                              #,'inq_fi'\
                                              #,'inq_last_12m'\
                                              #,'home_ownership'\
                                              #,'purpose'\
                                              #,'emp_length'\
                                              ,'installment'\
                                              #,'total_rev_hi_lim'\
                                              ,'loan_amnt'\
                                              #,'loan_status'\
                                              #,'verification_status'\
                                              ,'total_pymnt'\
                                              ,'grade'\
                                              ,'home'\
                                              ,'emp_length'\
                                              ,'purpose'\
                                              ,'verification_status']

### Training with DecisionTree

In [ ]:
dt = DecisionTreeClassifier(featuresCol='***features',labelCol='indexedLabel')

In [ ]:
pipeline_dt = Pipeline().setStages([gradeIndexer,gradeOneHotEncoder,\
                                    homeIndexer,homeOneHotEncoder,\
                                    emp_lengthIndexer,emp_lengthOneHotEncoder,\
                                    purposeIndexer,purposeOneHotEncoder,\
                                    verification_statusIndexer,verification_statusOneHotEncoder,\
                                    labelIndexer,\
                                    featureAssembler,\
                                    dt])

In [ ]:
## Check for Imbalance Class
training_dt.groupBy('loan_status').count().orderBy(['loan_status']).show()

In [ ]:
training_dt.count()

In [ ]:
start_time_dt = t.time()

In [ ]:
model_dt = pipeline_dt.fit(training_dt)

In [ ]:
duration_dt = t.time() - start_time_dt

In [ ]:
print('ใช้เวลา Distributed Training {0} วินาที'.format(duration_dt))

In [ ]:
spark_fullLabel_func = udf(lambda x: "Fully Paid" if x == 0.0 else "Charged Off")

In [ ]:
result_dt = model_dt.transform(test_dt.dropna())\
.withColumn('full_prediction',spark_fullLabel_func(F.col('prediction')))

In [ ]:
## Check for Imbalance Class
result_dt.groupBy(['loan_status','indexedLabel']).count().show()

In [ ]:
result_dt.count()

In [ ]:
NUM_ALL_TESTING = result_dt.count()

### การประเมิน Charge-Off: TPR, FNR (Recall, Miss rate) 

In [ ]:
result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(F.col('loan_status') == 'Charged Off').show()

In [ ]:
NUM_ACTUAL_POSITIVE = result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(F.col('loan_status') == 'Charged Off').count()

In [ ]:
NUM_ACTUAL_POSITIVE

In [ ]:
#ตรวจสอบความถูกต้องจากการ Predict ตรงกับข้อมูลที่เป็น IndexLabel หรือไม่ (เนื่องจากมีการทำ indexer ค่าที่ได้จึงเป็น Numerical)

### Positive = Charged Off

In [ ]:
result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') != col('prediction'))).filter(F.col('loan_status') == 'Charged Off').show()

In [ ]:
FN = result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') != col('prediction'))).filter(F.col('loan_status') == 'Charged Off').count()

In [ ]:
(FN/NUM_ACTUAL_POSITIVE)*100

In [ ]:
result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') == col('prediction'))).filter(F.col('loan_status') == 'Charged Off').show()

In [ ]:
TP = result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') == col('prediction'))).filter(F.col('loan_status') == 'Charged Off').count()

In [ ]:
(TP/NUM_ACTUAL_POSITIVE)*100

### การประเมิน Fully Paid: TNR, FPR (Specificity, Fall-out) 

In [ ]:
result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(F.col('loan_status') == 'Fully Paid').show()

In [ ]:
NUM_ACTUAL_NEGATIVE = result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(F.col('loan_status') == 'Fully Paid').count()

In [ ]:
NUM_ACTUAL_NEGATIVE

### Negative = Fully Paid

In [ ]:
result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') != col('prediction'))).filter(F.col('loan_status') == 'Fully Paid').show()

In [ ]:
FP = result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') != col('prediction'))).filter(F.col('loan_status') == 'Fully Paid').count()

In [ ]:
(FP/NUM_ACTUAL_NEGATIVE)*100

In [ ]:
result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') == col('prediction'))).filter(F.col('loan_status') == 'Fully Paid').show()

In [ ]:
TN = result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') == col('prediction'))).filter(F.col('loan_status') == 'Fully Paid').count()

In [ ]:
(TN/NUM_ACTUAL_NEGATIVE)*100

### การประเมิน Accuracy

In [ ]:
NUM_ACTUAL_NEGATIVE + NUM_ACTUAL_POSITIVE

In [ ]:
(TN+TP)/(NUM_ACTUAL_NEGATIVE + NUM_ACTUAL_POSITIVE)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
evaluator_DT = MulticlassClassificationEvaluator(predictionCol="prediction",\
                                              labelCol='indexedLabel', metricName='accuracy')
evaluator_DT.evaluate(result_dt)*100

In [ ]:
#model_dt.write().overwrite().save('gs://grizzy-lab/loanpayment_dtModel')

### ------------- สิ้นสุด Decision Tree ---------------------------

### Training with RandomForest

In [ ]:
RF = RandomForestClassifier(featuresCol='***features',labelCol='indexedLabel')

In [ ]:
pipeline_RF = Pipeline().setStages([gradeIndexer,gradeOneHotEncoder,\
                                    homeIndexer,homeOneHotEncoder,\
                                    emp_lengthIndexer,emp_lengthOneHotEncoder,\
                                    purposeIndexer,purposeOneHotEncoder,\
                                    verification_statusIndexer,verification_statusOneHotEncoder,\
                                    labelIndexer,\
                                    featureAssembler,\
                                    RF])

In [ ]:
#training_RF, test_RF = normalized_df.randomSplit([0.6,0.4])

In [ ]:
training_RF = training_dt

In [ ]:
test_RF = test_dt

In [ ]:
training_RF.groupBy('loan_status').count().orderBy(['loan_status']).show()

In [ ]:
training_RF.count()

In [ ]:
start_time_rf = t.time()

In [ ]:
model_RF = pipeline_RF.fit(training_RF)

In [ ]:
duration_rf = t.time() - start_time_rf

In [ ]:
print('ใช้เวลา Distributed Training {0} วินาที'.format(duration_rf))

In [ ]:
result_RF = model_RF.transform(test_RF.dropna()).withColumn('full_prediction',spark_fullLabel_func(F.col('prediction')))

In [ ]:
result_RF.groupBy('loan_status').count().show()

In [ ]:
result_RF.count()

In [ ]:
result_RF

### การประเมิน Charge-Off: TPR, FNR (Recall, Miss rate) 

In [ ]:
result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(F.col('loan_status') == 'Charged Off').show()

In [ ]:
NUM_ACTUAL_POSITIVE = result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(F.col('loan_status') == 'Charged Off').count()

In [ ]:
NUM_ACTUAL_POSITIVE

In [ ]:
#ตรวจสอบความถูกต้องจากการ Predict ตรงกับข้อมูลที่เป็น IndexLabel หรือไม่ (เนื่องจากมีการทำ indexer ค่าที่ได้จึงเป็น Numerical)

### Positive = Charged Off

In [ ]:
result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') != col('prediction'))).filter(F.col('loan_status') == 'Charged Off').show()

In [ ]:
FN = result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') != col('prediction'))).filter(F.col('loan_status') == 'Charged Off').count()

In [ ]:
FN

In [ ]:
(FN/NUM_ACTUAL_POSITIVE)*100

In [ ]:
result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') == col('prediction'))).filter(F.col('loan_status') == 'Charged Off').show()

In [ ]:
TP = result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') == col('prediction'))).filter(F.col('loan_status') == 'Charged Off').count()

In [ ]:
(TP/NUM_ACTUAL_POSITIVE)*100

### การประเมิน Fully Paid: TNR, FPR (Specificity, Fall-out) 

In [ ]:
result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(F.col('loan_status') == 'Fully Paid').show()

In [ ]:
NUM_ACTUAL_NEGATIVE = result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(F.col('loan_status') == 'Fully Paid').count()

In [ ]:
NUM_ACTUAL_NEGATIVE

In [ ]:
#ตรวจสอบความถูกต้องจากการ Predict ตรงกับข้อมูลที่เป็น IndexLabel หรือไม่ (เนื่องจากมีการทำ indexer ค่าที่ได้จึงเป็น Numerical)

### Negative = Fully Paid

In [ ]:
result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') != col('prediction'))).filter(F.col('loan_status') == 'Fully Paid').show()

In [ ]:
FP = result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') != col('prediction'))).filter(F.col('loan_status') == 'Fully Paid').count()

In [ ]:
(FP/NUM_ACTUAL_NEGATIVE)*100

In [ ]:
result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') == col('prediction'))).filter(F.col('loan_status') == 'Fully Paid').show()

In [ ]:
TN = result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') == col('prediction'))).filter(F.col('loan_status') == 'Fully Paid').count()

In [ ]:
(TN/NUM_ACTUAL_NEGATIVE)*100

### การประเมิน Accuracy

In [ ]:
NUM_ACTUAL_NEGATIVE + NUM_ACTUAL_POSITIVE

In [ ]:
(TN+TP)/(NUM_ACTUAL_NEGATIVE + NUM_ACTUAL_POSITIVE)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
evaluator_RF = MulticlassClassificationEvaluator(predictionCol="prediction",\
                                              labelCol='indexedLabel', metricName='accuracy')
evaluator_RF.evaluate(result_RF)*100

### ------------- สิ้นสุด Random Forest ---------------------------

### Training with Multi layer Perceptron (ANN)

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [ ]:
pipeline_mlp = Pipeline(stages=[gradeIndexer,gradeOneHotEncoder,\
                                    homeIndexer,homeOneHotEncoder,\
                                    emp_lengthIndexer,emp_lengthOneHotEncoder,\
                                    purposeIndexer,purposeOneHotEncoder,\
                                    verification_statusIndexer,verification_statusOneHotEncoder,\
                                    labelIndexer,\
                                    featureAssembler])

In [ ]:
#training_mlp, test_mlp = normalized_df.randomSplit([0.6,0.4])

In [ ]:
training_mlp = training_dt

In [ ]:
test_mlp = test_dt

In [ ]:
train_df_features = pipeline_mlp.fit(training_mlp).transform(training_mlp)

In [ ]:
layers = [train_df_features.schema["***features"].metadata["ml_attr"]["num_attrs"],20 ,10, 2]

In [ ]:
layers

### Implementation of MLP

In [ ]:
clf = MultilayerPerceptronClassifier(labelCol='indexedLabel'\
                                    ,featuresCol='***features'\
                                    ,layers = layers)

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [ ]:
pipeline_mlp_test = Pipeline(stages=[gradeIndexer,gradeOneHotEncoder,\
                                    homeIndexer,homeOneHotEncoder,\
                                    emp_lengthIndexer,emp_lengthOneHotEncoder,\
                                    purposeIndexer,purposeOneHotEncoder,\
                                    verification_statusIndexer,verification_statusOneHotEncoder,\
                                    labelIndexer,\
                                    featureAssembler,
                                    clf])

In [ ]:
#training_mlp_test, test_mlp_test = normalized_df.randomSplit([0.6,0.4])

In [ ]:
training_mlp.groupBy('loan_status').count().orderBy(['loan_status']).show()

In [ ]:
training_mlp.count()

In [ ]:
start_time_mlp = t.time()

In [ ]:
model_mlp = pipeline_mlp_test.fit(training_mlp)

In [ ]:
duration_mlp = t.time() - start_time_mlp

In [ ]:
print('ใช้เวลา Distributed Training {0} วินาที'.format(duration_mlp))

In [ ]:
result_mlp = model_mlp.transform(test_mlp.dropna())\
.withColumn('full_prediction',spark_fullLabel_func(F.col('prediction')))

In [ ]:
result_mlp.groupBy('loan_status').count().show()

In [ ]:
result_mlp.count()

In [ ]:
result_mlp

### การประเมิน Charge-Off: TPR, FNR (Recall, Miss rate) 

In [ ]:
result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(F.col('loan_status') == 'Charged Off').show()

In [ ]:
NUM_ACTUAL_POSITIVE = result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(F.col('loan_status') == 'Charged Off').count()

In [ ]:
NUM_ACTUAL_POSITIVE

In [ ]:
#ตรวจสอบความถูกต้องจากการ Predict ตรงกับข้อมูลที่เป็น IndexLabel หรือไม่ (เนื่องจากมีการทำ indexer ค่าที่ได้จึงเป็น Numerical)

### Positive = Charged Off

In [ ]:
result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') != col('prediction'))).filter(F.col('loan_status') == 'Charged Off').show()

In [ ]:
FN = result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') != col('prediction'))).filter(F.col('loan_status') == 'Charged Off').count()

In [ ]:
FN

In [ ]:
(FN/NUM_ACTUAL_POSITIVE)*100

In [ ]:
result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') == col('prediction'))).filter(F.col('loan_status') == 'Charged Off').show()

In [ ]:
TP = result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') == col('prediction'))).filter(F.col('loan_status') == 'Charged Off').count()

In [ ]:
(TP/NUM_ACTUAL_POSITIVE)*100

### การประเมิน Fully Paid: TNR, FPR (Specificity, Fall-out) 

In [ ]:
result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(F.col('loan_status') == 'Fully Paid').show()

In [ ]:
NUM_ACTUAL_NEGATIVE = result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(F.col('loan_status') == 'Fully Paid').count()

In [ ]:
NUM_ACTUAL_NEGATIVE

In [ ]:
#ตรวจสอบความถูกต้องจากการ Predict ตรงกับข้อมูลที่เป็น IndexLabel หรือไม่ (เนื่องจากมีการทำ indexer ค่าที่ได้จึงเป็น Numerical)

### Negative = Fully Paid

In [ ]:
result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') != col('prediction'))).filter(F.col('loan_status') == 'Fully Paid').show()

In [ ]:
FP = result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') != col('prediction'))).filter(F.col('loan_status') == 'Fully Paid').count()

In [ ]:
(FP/NUM_ACTUAL_NEGATIVE)*100

In [ ]:
result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') == col('prediction'))).filter(F.col('loan_status') == 'Fully Paid').show()

In [ ]:
TN = result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((F.col('indexedLabel') == col('prediction'))).filter(F.col('loan_status') == 'Fully Paid').count()

In [ ]:
(TN/NUM_ACTUAL_NEGATIVE)*100

### การประเมิน Accuracy

In [ ]:
NUM_ACTUAL_NEGATIVE + NUM_ACTUAL_POSITIVE

In [ ]:
(TN+TP)/(NUM_ACTUAL_NEGATIVE + NUM_ACTUAL_POSITIVE)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
evaluator_RF = MulticlassClassificationEvaluator(predictionCol="prediction",\
                                              labelCol='indexedLabel', metricName='accuracy')
evaluator_RF.evaluate(result_mlp)*100

### ------------- สิ้นสุด MLP ---------------------------

In [ ]:
#model_mlp.save('./modelMLP_bestloanpayment')